In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

In [2]:
df=pd.read_csv("train_clean_data.csv")
df.head()

,Unnamed: 0,Loan_ID,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,gender_Male,married_Yes,education_Not Graduate,property_area_Semiurban,property_area_Urban,self_employed_Yes,Loan_status_Y
0,1,LP001003,1,4583,1508.0,128.0,360.0,1.0,1,1,0,0,0,0,0
1,2,LP001005,0,3000,0.0,66.0,360.0,1.0,1,1,0,0,1,1,1
2,3,LP001006,0,2583,2358.0,120.0,360.0,1.0,1,1,1,0,1,0,1
3,4,LP001008,0,6000,0.0,141.0,360.0,1.0,1,0,0,0,1,0,1
4,5,LP001011,2,5417,4196.0,267.0,360.0,1.0,1,1,0,0,1,1,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               517 non-null    int64  
 1   Loan_ID                  517 non-null    object 
 2   Dependents               517 non-null    object 
 3   ApplicantIncome          517 non-null    int64  
 4   CoapplicantIncome        517 non-null    float64
 5   LoanAmount               517 non-null    float64
 6   Loan_Amount_Term         517 non-null    float64
 7   Credit_History           517 non-null    float64
 8   gender_Male              517 non-null    int64  
 9   married_Yes              517 non-null    int64  
 10  education_Not Graduate   517 non-null    int64  
 11  property_area_Semiurban  517 non-null    int64  
 12  property_area_Urban      517 non-null    int64  
 13  self_employed_Yes        517 non-null    int64  
 14  Loan_status_Y            5

In [4]:
df.shape

(517, 15)

In [5]:
df['Loan_status_Y'].value_counts()

1    360
0    157
Name: Loan_status_Y, dtype: int64

In [6]:

X=df.drop(['Loan_ID','Dependents','Loan_status_Y'],axis=1)
y=df['Loan_status_Y']

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test=train_test_split(X,y, test_size=0.33,stratify=y,random_state=42)


In [8]:
!pip install --upgrade scikit-learn
!pip install --upgrade imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [9]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [10]:
X_train_res.shape

(482, 12)

In [11]:
y_train_res.shape


(482,)

In [12]:

model1=DecisionTreeClassifier(class_weight='balanced')
model2=GaussianNB()
model3=LogisticRegression(solver='liblinear',max_iter=200,class_weight='balanced')

In [13]:
model1.fit(X_train_res,y_train_res)
model2.fit(X_train_res,y_train_res)
model3.fit(X_train_res,y_train_res)

LogisticRegression(class_weight='balanced', max_iter=200, solver='liblinear')

In [14]:
pred1=model1.predict_proba(X_test)
pred2=model2.predict_proba(X_test)
pred3=model3.predict_proba(X_test)

In [15]:
final_pred=(pred1+pred2+pred3)/3
final_pred

array([[0.37902581, 0.62097419],
       [0.75846954, 0.24153046],
       [0.02487486, 0.97512514],
       [0.03856742, 0.96143258],
       [0.34753724, 0.65246276],
       [0.22451861, 0.77548139],
       [0.26776946, 0.73223054],
       [0.09808941, 0.90191059],
       [0.14379217, 0.85620783],
       [0.08879699, 0.91120301],
       [0.45635843, 0.54364157],
       [0.14846579, 0.85153421],
       [0.05662628, 0.94337372],
       [0.11200459, 0.88799541],
       [0.10200454, 0.89799546],
       [0.16371618, 0.83628382],
       [0.04855579, 0.95144421],
       [0.03929299, 0.96070701],
       [0.9797375 , 0.0202625 ],
       [0.07566218, 0.92433782],
       [0.79497078, 0.20502922],
       [0.27369718, 0.72630282],
       [0.03752346, 0.96247654],
       [0.06249971, 0.93750029],
       [0.61430916, 0.38569084],
       [0.42405506, 0.57594494],
       [0.0195967 , 0.9804033 ],
       [0.16976034, 0.83023966],
       [0.08663018, 0.91336982],
       [0.27300562, 0.72699438],
       [0.

In [16]:
model1.classes_

array([0, 1], dtype=int64)

In [17]:
clf1=DecisionTreeClassifier(class_weight='balanced')
clf2=GaussianNB()
clf3=LogisticRegression(solver='liblinear',max_iter=200,class_weight='balanced')


In [18]:

eclf1=VotingClassifier(estimators=[('DT',clf1),('GNB',clf2),('LR',clf3)], voting='soft')
eclf1=eclf1.fit(X_train_res,y_train_res)


In [19]:

predictions=eclf1.predict(X_test)
predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0], dtype=int64)

In [20]:
from sklearn.metrics import classification_report , confusion_matrix

print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.66      0.60      0.63        52
           1       0.83      0.87      0.85       119

    accuracy                           0.78       171
   macro avg       0.75      0.73      0.74       171
weighted avg       0.78      0.78      0.78       171

[[ 31  21]
 [ 16 103]]


In [21]:
predictions.shape

(171,)

In [22]:
eclf1=VotingClassifier(estimators=[('DT',clf1),('GNB',clf2),('LR',clf3)], voting='hard')
eclf1=eclf1.fit(X_train_res,y_train_res)


In [23]:
from sklearn.metrics import classification_report , confusion_matrix

print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.66      0.60      0.63        52
           1       0.83      0.87      0.85       119

    accuracy                           0.78       171
   macro avg       0.75      0.73      0.74       171
weighted avg       0.78      0.78      0.78       171

[[ 31  21]
 [ 16 103]]


In [24]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
base_model1=DecisionTreeClassifier()
bagging = BaggingClassifier(estimator=base_model1, n_estimators=200, random_state=0)
bagging.fit(X_train_res, y_train_res)
y_pred_bagging = bagging.predict(X_test)
print("Bagging Accuracy:", accuracy_score(y_test, y_pred_bagging))

Bagging Accuracy: 0.7953216374269005


In [25]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=0)
rf.fit(X_train_res, y_train_res)
y_pred_rf = rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))


Random Forest Accuracy: 0.783625730994152


In [27]:

from sklearn.ensemble import AdaBoostClassifier

boosting = AdaBoostClassifier(n_estimators=200, random_state=0)
boosting.fit(X_train_res, y_train_res)
y_pred_boost = boosting.predict(X_test)
print("Boosting (AdaBoost) Accuracy:", accuracy_score(y_test, y_pred_boost))


Boosting (AdaBoost) Accuracy: 0.7719298245614035
